In [30]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import pandas as pd
from langchain_text_splitters import RecursiveCharacterTextSplitter

file_path = ('../data/edgar_filings2.csv')

loader = CSVLoader(
    file_path=file_path,
    encoding="utf-8",
    csv_args={
        "delimiter": "|",
        "quotechar": '"'
    },
    metadata_columns=["company", 'date', 'section']
)
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

texts=text_splitter.split_documents(data)

test_texts=texts[0:10]

In [31]:
test_texts

[Document(metadata={'source': '../data/edgar_filings2.csv', 'row': 0, 'company': 'HUBS', 'date': '2024-02-14', 'section': 'business'}, page_content='text: page no. part i item 1. business 7 item 1a. risk factors 16 item 1b. unresolved staff comments 43 item 1c. cybersecurity item 2. properties item 3. legal proceedings 44 item 4. mine safety disclosures part ii item 5. market for registrant’s common equity, related stockholder matters and issuer purchases of equity securities item 6. [reserved] item 7. management’s discussion and analysis of financial condition and results of operations item 7a. quantitative and qualitative disclosures about market risk item 8. financial statements and supplementary data item 9. changes in and disagreements with accountants on accounting and financial disclosures 97 item 9a. controls and procedures item 9b. other information item 9c. disclosure regarding foreign jurisdictions that prevent inspections part iii item 10. directors, executive officers and 

In [35]:
from langchain_openai import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-small",
    azure_endpoint='https://aiall9596864698.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2023-05-15',
    api_key='6d440529fad24ffc8aef6d5f9ef52593',
    openai_api_version="2024-07-01-preview"
)

In [14]:
from langchain_chroma import Chroma

vector_store = Chroma(
    collection_name="test_collection",
    embedding_function=embeddings,
    persist_directory="../data/chroma_langchain_db_test"
)

2024-11-26 01:22:19,021 - INFO - Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


In [37]:
from langchain_core.vectorstores import InMemoryVectorStore

text = "LangChain is the framework for building context-aware reasoning applications"

vectorstore = InMemoryVectorStore.from_texts(
    [text],
    embedding=embeddings,
)

2024-12-03 00:19:35,026 - INFO - HTTP Request: POST https://aiall9596864698.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-07-01-preview "HTTP/1.1 200 OK"


In [15]:
from uuid import uuid4
uuids = [str(uuid4()) for _ in range(len(test_texts))]
vector_store.add_documents(documents=test_texts, ids=uuids)

2024-11-26 01:23:46,885 - INFO - HTTP Request: POST https://aiall9596864698.cognitiveservices.azure.com/openai/deployments/text-embedding-3-small/embeddings?api-version=2024-07-01-preview "HTTP/1.1 200 OK"


['01187911-03e8-4f18-85f6-96b55b6f61ca',
 'a8ce9964-a8d1-4941-b6f4-b90cf9afdd39',
 '38e339a4-58f6-4479-9d48-e3358f66e6be',
 '8d87771f-b249-48f6-9a3d-fb214796cd1c',
 '5f6d0a14-b1a6-47e9-a6f2-917b5bedaf1f',
 '3137dca4-efab-4fa4-8774-80226c82b262',
 'f514500d-158a-47ff-a584-d1d4c115d802',
 '2d1326ce-79bf-4572-9f18-1a8c10021609',
 '729fded3-f747-4cad-ad7f-0a4153bc6c2d',
 'd11e5d12-ae35-4395-960c-375ef42c2d21']

In [25]:
import langchain
import langgraph
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool
import langchain_core
import langchain_openai
from langchain_openai import ChatOpenAI
from langchain.agents import Tool, AgentExecutor, create_react_agent, create_tool_calling_agent, initialize_agent, create_openai_tools_agent
from langchain_core.output_parsers.openai_functions import JsonOutputFunctionsParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage
from langgraph.graph import END, StateGraph, START
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain.tools import BaseTool
from langchain_core.agents import AgentAction, AgentFinish
import orjson
import json
import os
import openai
from typing import Literal, Union, Sequence, TypedDict, Annotated
import pandas as pd
from openai import OpenAI
import functools
import operator
import logging
import streamlit as st
from langchain_openai import AzureChatOpenAI
from typing import Optional
from langchain.retrievers.self_query.chroma import ChromaTranslator
from langchain.chains.query_constructor.ir import Comparator, Comparison, Operation, Operator, StructuredQuery
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_chroma import Chroma
from langchain.tools.retriever import create_retriever_tool

os.environ["AZURE_OPENAI_API_KEY"]="b229fdc63a3d4b06b4adcc67660474f3"
os.environ["AZURE_OPENAI_ENDPOINT"]="https://openai-541.openai.azure.com/"
os.environ["ALPHAVANTAGE_API_KEY"]="5TW2G5K4N9GKMGRD"

pd.set_option("display.max_colwidth", None)
pd.set_option('display.max_rows', None)
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
#logging.disable(logging.CRITICAL)

import sys
import os
from io import StringIO

original_stdout = sys.stdout  # Save a reference to the original stdout
captured_output = StringIO()  # Create a new StringIO object


In [28]:
class ChromaRetrieverTool:
    def __init__(self, directory, collection, embedding, llm):
        self.directory = directory
        self.collection = collection
        self.embedding = embedding
        self.llm = llm

    def initiate_retriever(self):
        metadata_field_info = [
            AttributeInfo(
                name="date",
                description="The date of 10-k filing",
                type="string or list[string]",
            ),
            AttributeInfo(
                name="section",
                description="The section of the 10-k filing",
                type="string or list[string]",
            ),
            AttributeInfo(
                name="company",
                description="The name of the company",
                type="string or list[string]",
            ),
            ]
        
        document_content_description = "Chunked content of the 10-k filing"

        self.vector_store = Chroma(
            collection_name=self.collection,
            persist_directory=self.directory,
            embedding_function=self.embedding
        )

        self.retriever=SelfQueryRetriever.from_llm(
            self.llm,
            self.vector_store,
            document_content_description,
            metadata_field_info,
            verbose=True
        )

        self.retriever_tool=create_retriever_tool(
            self.retriever,
            '10-k document retriever',
            'Query a retriever to get 10-k document information',
        )

class AgentState(TypedDict):
    """State dictionary for the agent.
    
    Holds the messages and the next state to transition to.
    """
    messages: Annotated[Sequence[BaseMessage], operator.add]
    next: str
    #intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

def create_agent(llm: AzureChatOpenAI, tools: list, system_prompt: str):
    """Creates an agent using the provided LLM and tools.
    
    Sets up a prompt template and creates an agent executor.
    
    :param llm: Language model to be used by the agent.
    :param tools: List of tools available to the agent.
    :param system_prompt: System-level prompt for the agent.
    :return: AgentExecutor instance configured with the provided settings.
    """
    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder(variable_name="messages"),
            MessagesPlaceholder(variable_name="agent_scratchpad"),
        ]
    )
    agent = create_openai_tools_agent(llm, tools, prompt)

    executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
    
    return executor

def agent_node(state, agent, name):
    """Executes the agent with the given state and returns the result.
    
    Handles errors and formats the result into a message.
    
    :param state: Current state of the agent.
    :param agent: Agent instance to be executed.
    :param name: Name of the agent for identifying the source of the message.
    :return: Dictionary containing the result message.
    """
    try:
        result = agent.invoke(state)

        return {"messages": [HumanMessage(content=result["output"], name=name)]}
    except Exception as e:
        print(f"Error with agent: {e}")
        return {"messages": [HumanMessage(content="Error occurred", name=name)]}
    
def create_llm(
    model: str = "gpt-35-turbo",
    api_version: str = "2024-07-01-preview",
    temperature: float = 0,
    max_tokens: int = 500,
    timeout: int = None,
    max_retries: int = 2
) -> AzureChatOpenAI:
    """Creates an instance of ChatOpenAI with specified parameters.
    
    :param model: The model to use (default is "gpt-4o").
    :param temperature: Controls the randomness of the output (default is 0).
    :param max_tokens: Maximum number of tokens in the output (default is 500).
    :param timeout: Timeout duration for the request (default is None).
    :param max_retries: Number of retry attempts in case of failure (default is 2).
    :return: An instance of ChatOpenAI configured with the given parameters.
    """
    try:
        
        llm = AzureChatOpenAI(
            azure_deployment=model,  # or your deployment
            api_version=api_version,  # or your api version
            temperature=0,
            max_tokens=None,
            timeout=None,
            max_retries=2,
        )

        return llm
    
    except Exception as e:
        # Log error message and re-raise exception for further handling if needed
        print(f"Error with LLM creation: {e}")
        raise

class AgentManager:
    def __init__(self, llm):
        """
        Initialize the AgentManager with the given language model.
        
        Args:
            llm: The language model instance to use for creating agents.
        """
        self.llm = llm
        
        # Initialize tools for different agents
        chroma_retriever = ChromaRetrieverTool(directory="../data/chroma_langchain_db_test", collection="test_collection", embedding=embeddings, llm=create_llm())
        chroma_retriever.initiate_retriever()
        self.tenk_tools = [chroma_retriever.retriever_tool]
        
        # Create the agents
        self.create_agents()
        
    def create_agents(self):
        """
        Create agents for Snowflake table operations, SQL querying, and documentation searching.
        """
        # Create Snowflake Expert Agent
        tenk_agent_system_prompt = (
            "You are an expert in 10-k filings. You can provide information about a company's 10-k filings."
        )
        self.tenk_agent = create_agent(
            self.llm, self.tenk_tools, tenk_agent_system_prompt
        )
        self.tenk_node = functools.partial(agent_node, agent=self.tenk_agent, name="Ten-K-Expert")
        




        # Create SQL Coder Agent
        #sql_agent_tools = [SnowparkQueryTool(snowpark_adapter=self.snowpark_adapter)]
        #sql_agent_system_prompt = "You are a Snowflake SQL expert."
        #self.sql_agent = create_agent(self.llm, sql_agent_tools, sql_agent_system_prompt)
        #self.sql_node = functools.partial(agent_node, agent=self.sql_agent, name="SQL-Coder")
        
        # Create Documentation Agent
        #documentation_agent_system_prompt = "You are an expert in the company's documentation."
        #self.documentation_agent = create_agent(self.llm, self.documentation_tools, documentation_agent_system_prompt)
        #self.documentation_node = functools.partial(agent_node, agent=self.documentation_agent, name="Documentation-Expert")

        
    def get_nodes(self):
        """
        Retrieve the nodes for each agent.
        
        Returns:
            dict: A dictionary where keys are agent names and values are their corresponding nodes.
        """
        return {
            "Ten-K-Expert": self.tenk_node,
            #"SQL-Coder": self.sql_node,
            #"Snowflake-Expert": self.snowflake_node
        }

class WorkflowManager:
    def __init__(self, agent_manager):
        """
        Initialize the WorkflowManager with the given AgentManager.
        
        Args:
            agent_manager: The instance of AgentManager to manage the workflow.
        """
        self.agent_manager = agent_manager
        self.workflow = StateGraph(AgentState)
        self.setup_workflow()
    
    def setup_workflow(self):
        """
        Set up the workflow by adding nodes, configuring the supervisor, defining edges, and compiling the graph.
        """
        # Add nodes for each agent to the workflow
        for role, node in self.agent_manager.get_nodes().items():
            self.workflow.add_node(role, node)
        
        # Add the supervisor node to the workflow
        self.setup_supervisor()
        
        # Define the edges in the workflow
        self.define_edges()
        
        # Compile the graph to finalize the workflow setup
        self.graph = self.workflow.compile()
    
    def setup_supervisor(self):
        """
        Set up the supervisor node to manage the conversation between agents.
        """
        members = list(self.agent_manager.get_nodes().keys())
        options = ["FINISH"] + members
        system_prompt = (
            "You are a supervisor tasked with managing a conversation between the "
            "following workers: {members}. Then respond with the worker to act next. Each worker will perform a task and respond with their results "
            "and status. Continue this until you have enough information."
        )
        
        supervisor_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", system_prompt),
                MessagesPlaceholder(variable_name="messages"),
                (
                    "system",
                    "Given the conversation above, who should act next? "
                    "Or should we FINISH? Summarize the conversation always relating it to the original question then Select one of: {options}.",
                ),
            ]
        ).partial(options=str(options), members=", ".join(members))
        
        # Define function for routing based on supervisor's decision
        function_def = {
            "name": "route",
            "description": "Select the next role.",
            "parameters": {
                "title": "routeSchema",
                "type": "object",
                "properties": {
                    "next": {
                        "title": "Next",
                        "anyOf": [{"enum": options}],
                    }
                },
                "required": ["next"],
            },
        }
        
        self.supervisor_chain = (
            supervisor_prompt
            | self.agent_manager.llm.bind_functions(functions=[function_def], function_call="route")
            | JsonOutputFunctionsParser()
        )
        # Add the supervisor chain as a node in the workflow
        self.workflow.add_node("supervisor", self.supervisor_chain)
    
    def define_edges(self):
        """
        Define the edges between nodes in the workflow.
        """
        members = list(self.agent_manager.get_nodes().keys())
        # Add edges from each member to the supervisor
        for member in members:
            self.workflow.add_edge(member, "supervisor")
        
        # Map member names to corresponding nodes and define conditional edges
        conditional_map = {k: k for k in members}
        conditional_map["FINISH"] = END
        self.workflow.add_conditional_edges("supervisor", lambda x: x["next"], conditional_map)
        # Add edge from the start node to the supervisor
        self.workflow.add_edge(START, "supervisor")

def my_query(myquery, graph):
    """
    Initiates the workflow graph with the provided query and streams the results.

    This function takes a user query, feeds it into the specified workflow graph, and processes the results as they are produced. 
    It prints each step of the result until the workflow reaches the end.

    Args:
        myquery (str): The user query to be processed by the workflow graph.
        graph (StateGraph): The workflow graph instance that manages the execution and processing of the query.

    Usage:
        Call this function with a query string and a graph instance to start the workflow and see the intermediate results.

    Example:
        my_query("change password", graph_instance)
    """
    # Stream the results of the graph execution based on the provided query

    final_state=graph.invoke(
        {"messages": [HumanMessage(content=myquery)]},
        {"recursion_limit": 100},
        #return_intermediate_steps=True,
    )

    llm = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo",  # or your deployment
    api_version="2024-07-01-preview",  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    )

    
    
    
    #st.session_state.messages.append({"text": final.content, "role": "assistant"})

    #st.markdown(final.content)
    #st.markdown(final_state["messages"][-1].content)

    return final_state, final






def main():
    
    """
    Main function to execute the workflow.
    
    This function sets up a Snowflake connection, initializes components such as the SQL adapter 
    and language model, creates agents and workflow, and then executes a query.
    """
    st.title("Ask Questions to your Own Personal Data Assistant:")
    st.write("""Ask your questions here.""")
    

    if 'messages' not in st.session_state:
        st.session_state.messages = []

    # Initialize components
    llm = create_llm()
    
    if not llm:
        # Log error if LLM instance creation fails
        logging.error("Failed to create LLM instance.")
        st.error("Failed to create LLM instance.")
        return

    # Create agents and workflow
    try:
        # Initialize AgentManager with LLM and SQL adapter
        agent_manager = AgentManager(llm)
        # Initialize WorkflowManager with the created agents
        workflow_manager = WorkflowManager(agent_manager)
        # Access the compiled workflow graph
        graph = workflow_manager.graph

    except Exception as e:
        # Log any exceptions that occur during agent or workflow setup
        logging.error(f"An error occurred while setting up agents or workflow: {e}")
        return

    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
    
    # React to user input
    if myquery := st.chat_input("What is up?"):
        # Display user message in chat message container
        st.chat_message("user").markdown(myquery)
        # Add user message to chat history
        st.session_state.messages.append({"role": "user", "content": myquery})
    
        try:
            # Execute the query and process results
            final_state, final=my_query(myquery, graph)
        except Exception as e:
            # Log any exceptions that occur during query execution
            logging.error(f"An error occurred while executing the query: {e}")
        
    
        response = f"Bot: {final.content}"
        # Display assistant response in chat message container
        with st.chat_message("assistant"):
            st.markdown(response)
        
        st.session_state.messages.append({"role": "assistant", "content": response})
        
        return final_state, final

if __name__ == "__main__":
    sys.stdout = captured_output


    final_state=main()
    sys.stdout = original_stdout
    output = captured_output.getvalue()

    with st.expander("Verbose Output", expanded=False):
        if output:
            st.text(output)  # or st.write(output) for more flexibility
    with st.expander("Agent Conversation", expanded=False):
        if final_state:
            for i in final_state[0]["messages"]:
                st.text(i.pretty_repr())  # Display each message as plain text





2024-12-01 20:05:12.883 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-01 20:05:12.891 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-01 20:05:12.891 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-01 20:05:12.891 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-01 20:05:12.899 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-01 20:05:12.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-01 20:05:12.907 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
C:\Users\Dave\AppData\Local\Temp\ipykernel_10252\1682207930.py:266: LangChainDeprecationWarning: The method `BaseChatO

In [4]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="gpt-35-turbo",  # or your deployment
    api_version="2024-07-01-preview",  # or your api version
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)